In [54]:

# coding: utf-8

# In[3]:


from __future__ import print_function

import sys
import numpy as np
from time import time
import matplotlib.pyplot as plt 

from PIL import Image

sys.path.append('/home/xilinx')
from pynq import Overlay
from pynq import allocate

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))

    print("Start of \"" + sys.argv[0] + "\"")

    ol = Overlay("/home/xilinx/r09525031/xapp890/design_1_wrapper.bit")
    Sobel_filter = ol.sobel_filter_0


    fiSamples = open("/home/xilinx/r09525031/xapp890/test_1080p.dat", "rb+")

    max_row = 1080;
    max_col = 1920;

    length = max_row*max_col;
    
    input_pixel = allocate(shape=(max_row,max_col), dtype=np.int32)
    output_pixel = allocate(shape=(max_row,max_col), dtype=np.int32)
  
    fiSamples.seek(0)
    for j in range(max_col):
        for i in range(max_row):
            line = fiSamples.readline();
            tmpB = int(line);
            line = fiSamples.readline();
            tmpG = int(line);
            line = fiSamples.readline();
            tmpR = int(line);
            input_pixel[i][j] = tmpB*65536 + tmpG*256 + tmpR;
    
    fiSamples.close()
    
    
    Sobel_filter.write(0x10, input_pixel.device_address);
    Sobel_filter.write(0x18, output_pixel.device_address);
    Sobel_filter.write(0x20,max_row);
    Sobel_filter.write(0x28,max_col);

    print("Start");
    timeKernelStart = time()
    
    Sobel_filter.write(0x00, 0x01);
    
    timeKernelEnd = time();
    print("Kernel execution time: " + str(timeKernelEnd - timeKernelStart) + " s");
    
#     fout = open("/home/xilinx/r09525031/xapp890/out.dat", "w+")
    
    print("Image Gen");
    img = Image.new("RGB",(max_row, max_col));
    pix = img.load();
    for j in range(max_col):
        for i in range(max_row):
#             img.putpixel((i,j),((output_pixel & 0x0000FF),(output_pixel & 0x00FF00)>>8),(output_pixel & 0xFF0000)>>16))
            tmp_r = (output_pixel[i][j] & 0x0000FF);
            tmp_g = (output_pixel[i][j] & 0x00FF00)>>8;
            tmp_b = (output_pixel[i][j] & 0xFF0000)>>16;
            pix[i,j] = (tmp_r,tmp_g,tmp_b);
            
    img.save("/home/xilinx/r09525031/xapp890/hw_out.bmp","BMP");
    
    
#     fout.close();


    print("============================")
    print("Exit process")



Entry: /usr/lib/python3/dist-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/lib/python3/dist-packages/ipykernel_launcher.py"
Start
Kernel execution time: 5.817413330078125e-05 s
Image Gen
Exit process
